## Цель исследования

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Наша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Моя первая задача как аналитика — проанализировать базу данных.

В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.



In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
              'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
               'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 


## Исследование таблиц — выведение первых строк

In [2]:
# Формируем sql-запрос, который выгрузит нужные нам поля.
query = """
SELECT *
FROM books
"""

In [3]:
books = pd.io.sql.read_sql(query, con = engine) 
books.head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

In [4]:
query = """
SELECT *
FROM authors
"""

In [5]:
authors = pd.io.sql.read_sql(query, con = engine) 
authors.head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

In [6]:
query = """
SELECT *
FROM publishers
"""

In [7]:
publishers = pd.io.sql.read_sql(query, con = engine) 
publishers.head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

In [8]:
query = """
SELECT *
FROM ratings
"""

In [9]:
ratings = pd.io.sql.read_sql(query, con = engine) 
ratings.head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

In [10]:
query = """
SELECT *
FROM reviews
"""

In [11]:
reviews = pd.io.sql.read_sql(query, con = engine) 
reviews.head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

## Задания

### Посчитать, сколько книг вышло после 1 января 2000 года

In [12]:
query = """
SELECT COUNT(book_id)
FROM books
WHERE publication_date > '2000-01-01' 
"""

In [13]:
answer1 = pd.io.sql.read_sql(query, con = engine) 
answer1.head()

,count
0,819


#### Вывод:

После 1 января 2000 года вышло 819 книг

### Для каждой книги посчитать количество обзоров и среднюю оценку

In [14]:
query = """
SELECT b.book_id,
       COUNT(DISTINCT re.review_id) cnt_review,
       AVG(ra.rating) avg_rating
FROM books b  
LEFT JOIN reviews re ON b.book_id=re.book_id 
LEFT JOIN ratings ra ON re.book_id=ra.book_id 
GROUP BY 1
ORDER BY 2,3
"""

In [15]:
answer2 = pd.io.sql.read_sql(query, con = engine) 
answer2

,book_id,cnt_review,avg_rating
0,672,0,NaN
1,808,0,NaN
2,387,0,NaN
3,221,0,NaN
4,83,0,NaN
...,...,...,...
995,734,6,4.206897
996,656,6,4.264151
997,299,6,4.287500
998,302,6,4.414634


#### Вывод:

Вывели количество обзоров и среднюю оценку на каждую книгу
* Всех больше оставили обзоров на книгу - 7 раз
* Всех меньше обзоров на книгу - 1 раза


### Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры;

In [16]:
query = """
SELECT p.publisher_id,
       p.publisher,
       COUNT(b.book_id) cnt_book
FROM publishers p   
JOIN books b ON p.publisher_id=b.publisher_id 
WHERE num_pages>50
GROUP BY 1,2
ORDER BY 3 DESC
LIMIT 1
"""

In [17]:
answer3 = pd.io.sql.read_sql(query, con = engine) 
answer3

,publisher_id,publisher,cnt_book
0,212,Penguin Books,42


#### Вывод:

Наибольшее число книг (42 книги) было выпущенно в издательстве Penguin Books.

### Определить автора с самой высокой средней оценкой книг — учитывать только книги с 50 и более оценками

In [18]:
query = """
SELECT a.author_id,
       a.author,
       AVG(r.rating)
FROM authors a   
JOIN books b ON a.author_id=b.author_id
JOIN ratings r ON b.book_id=r.book_id 
WHERE b.book_id in (SELECT b.book_id
                    FROM books b   
                    JOIN ratings r ON b.book_id=r.book_id 
                    group by 1
                    HAVING COUNT(r.rating)>=50)
GROUP BY 1,2                   
ORDER BY 3 DESC
LIMIT 1
"""

In [19]:
answer4 = pd.io.sql.read_sql(query, con = engine) 
answer4

,author_id,author,avg
0,236,J.K. Rowling/Mary GrandPré,4.287097


#### Вывод:

Самую высокую среднюю оценку книг имеют авторы J.K. Rowling/Mary GrandPré.

### Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [20]:
query = """
SELECT ROUND(AVG(d.cnt_review)) avg_cnt_rewiews
FROM (SELECT username,
     COUNT (review_id) cnt_review
     FROM reviews   
     GROUP BY 1) as d
WHERE username in (SELECT username
                   FROM ratings 
                   GROUP BY 1
                   HAVING COUNT(rating)>50)     
 """  

In [21]:
answer5 = pd.io.sql.read_sql(query, con = engine) 
answer5

,avg_cnt_rewiews
0,24.0


#### Вывод: 

Среднее количество обзоров равно 24, от пользователей, которые поставили больше 50 оценок

### Общий вывод

Проанализировав базу данных, получили следующие сведения:
* После 1 января 2000 года вышло 819 книг
* Количество обзоров на каждую книгу:
    * Всех больше оставили обзоров на книгу - 7 раз
    * Всех меньше обзоров на книгу - 1 раза
* Наибольшее число книг (42 книги) было выпущенно в издательстве Penguin Books.
* Самую высокую среднюю оценку книг имеют авторы J.K. Rowling/Mary GrandPré.
* Среднее количество обзоров равно 24, от пользователей, которые поставили больше 50 оценок